In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/datamining/processed_data.xlsx')

In [ ]:
# 수치형 데이터만 선택
numeric_df = df.select_dtypes(include=['number'])

# '몸값'과의 상관관계에서 특정 열('이름') 제외
print(numeric_df.corr()['몸값'].drop(['몸값']))

선형 구조 파악

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

비선형 구조임을 증명하는 근거 2

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# 1. 독립 변수와 종속 변수 설정

# 열 이름에서 공백 제거
df.columns = df.columns.str.strip()

X = df[['신장', '나이', '출전 시간', '득점', '슛', '어시스트', '예상 어시스트',
       '성공한 패스', '패스 정확도', '터치', '볼 뺏김', '볼 경합 성공', '경고', '퇴장']]
df['몸값'] = np.log1p(df['몸값'])
y = df['몸값']

# 종속 변수 로그 변환
y_log = np.log1p(y)  # log(1 + y) 변환

In [ ]:
# 2. 데이터 분할
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.4, random_state=42)

In [ ]:
# 3. 랜덤 포레스트 모델 생성 및 학습
rf_model = RandomForestRegressor(n_estimators= 200 # 트리 수 증가
                                 , max_depth = 10 # 트리 깊이 제한
                                 , min_samples_split = 5
                                 , random_state = 42)
rf_model.fit(X_train, y_train_log)

In [ ]:
# 5. 예측 및 지수 변환(로그 복원)
y_train_pred_log = rf_model.predict(X_train)
y_test_pred_log = rf_model.predict(X_test)
y_train_pred = np.expm1(y_train_pred_log)  # 예측값 복원
y_test_pred = np.expm1(y_test_pred_log)

In [ ]:
# 5. R^2 점수와 RMSE 계산
y_train_actual = np.expm1(y_train_log)  # 실제값 복원
y_test_actual = np.expm1(y_test_log)    # 실제값 복원

train_r2 = r2_score(y_train_actual, y_train_pred)
test_r2 = r2_score(y_test_actual, y_test_pred)

train_mse = mean_squared_error(y_train_actual, y_train_pred)
test_mse = mean_squared_error(y_test_actual, y_test_pred)

train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

In [ ]:
print(f"학습 데이터 R²: {train_r2:.3f}, RMSE: {train_rmse:.3f}")
print(f"테스트 데이터 R²: {test_r2:.3f}, RMSE: {test_rmse:.3f}")

In [ ]:
import pandas as pd # 피처별로 중요도를 나타냄.


importance = rf_model.feature_importances_
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)
print(feature_importance)

나이, 슛, 볼 경합 성공, 터치, 성공한 패스, 출전시간, 패스 정확도, 신장 순으로 중요도

옛날보다 수비수들도 빌드업에 많이 참여할수록 가치가 올라감. 생각보다 공격 포인트가 높을 수록 고평가를 받는 건 아닌 듯.

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
%matplotlib inline

데이터 시각화

In [ ]:
# '이름' 열 제거
try:
    df_no_name = df.drop(columns=['이름'])
except KeyError:
    df_no_name = df

# '몸값'과 다른 변수 간 상관관계 계산
target_corr = df_no_name.corr()['몸값'].drop('몸값')

# 히트맵 시각화
plt.figure(figsize=(6, 8))
sns.heatmap(target_corr.to_frame(), annot=True, cmap='coolwarm', fmt=".2f", cbar=True)
plt.title('종속 변수 "몸값"과 독립 변수 상관관계 히트맵', fontsize=14)
plt.xticks(rotation=0, fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
for feature in ['성공한 패스', '터치', '패스 정확도', '출전 시간', '득점', '나이']:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=df[feature], y=df['몸값'])
    plt.title(f'{feature} vs 몸값')
    plt.xlabel(feature)
    plt.ylabel('몸값')
    plt.show()

In [ ]:
import numpy as np

# 데이터
datasets = ['학습 데이터', '테스트 데이터']
r2_scores = [0.857, 0.209]
rmse_scores = [0.004, 0.010]

x = np.arange(len(datasets))

# 막대그래프
plt.figure(figsize=(10, 6))

# R²
plt.bar(x - 0.2, r2_scores, width=0.4, label='R²', color='blue')
# RMSE
plt.bar(x + 0.2, rmse_scores, width=0.4, label='RMSE', color='orange')

plt.xticks(x, datasets)
plt.ylabel("Score")
plt.title("R²와 RMSE 비교")
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()
